In [1]:
##import Modules

# Load Modules
import numpy as np
import astropy.units as u

# import plotting modules
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.colors import LogNorm
#from mpl_toolkits.mplot3d import Axes3D
#from matplotlib import cm
%matplotlib inline

# my modules
from ReadFile import Read
from CenterOfMass import CenterOfMass
from MassProfile import MassProfile

In [2]:
#what I want to work on the most right now is getting my sersic files to work at all snapshots

#end profile- snap 800

#COM Data for MW Disk
COM_MW = CenterOfMass("MW_VLowRes/MW_800.txt",2)
#COM Data for M31 Disk
COM_M31 = CenterOfMass("M31_VLowRes/M31_800.txt",2)

#COM position by Galaxy
#MW
COMP_MW = COM_MW.COM_P(0.1)
#M31
COMP_M31 = COM_M31.COM_P(0.1)

#just the position based on the coordinates from CenterofMass outplut
MWx = COM_MW.x-COMP_MW[0].value
MWy = COM_MW.y-COMP_MW[1].value
M31x = COM_M31.x-COMP_M31[0].value
M31y = COM_M31.y-COMP_M31[1].value

#Sersic Function
def SersicE(R,Re,n,Mtot):
#inputs:
    #R - the radius (units: kpc)
    #Re- the half light radius (units: kpc)
    #n - Sercic Index
    #Mtot - total stellar mass of the system (untis: Msun)
#Returns:
    #Surface brightness profile (units: Lsun/kpc^2)
    
    #assume mass to light = 1
    L = Mtot #total luminosity = total stellar mass
    
    #from equation above for Ie
    Ie = L/7.2/np.pi/Re**2
    
    #exponent term
    A = (R/Re)**(1/n)-1
    
    return Ie*np.exp(-7.67*A)


#add together
#Both_StellarI = np.concatenate((MW_StellarI,M31_StellarI),axis=0)


In [25]:
MW = MassProfile("MW_VLowRes/MW",405)
M31 = MassProfile("M31_VLowRes/M31",405)

#create an array of radii
r = np.arange(0.1,30.0,0.1)

#create mass profile of the stellar mass of both galaxies
#use particle type 2 for this then
MW_StellarMass = MW.MassEnclosed(2,r)
M31_StellarMass = M31.MassEnclosed(2,r)


#compute surface mass density profile, assuming M/L ~ 1
#I = L/4 pi D^2
#note- spherical, may not be appropriate for a disk
MW_StellarI = MW_StellarMass/4/np.pi/r**2
M31_StellarI = M31_StellarMass/4/np.pi/r**2

In [26]:
#compute half mass radius
#total mass of stellar disk
MW_StellarTotal = np.max(MW_StellarMass)
M31_StellarTotal = np.max(M31_StellarMass)
#find half mass radius by finding where mass drops by factor of 1/2
MW_Low = MW_StellarTotal/2
MW_High = MW_StellarTotal/2 + MW_StellarTotal/2*0.01
M31_Low = M31_StellarTotal/2
M31_High = M31_StellarTotal/2 + M31_StellarTotal/2*0.01

#index for both galaxies
index_MW = np.where((MW_StellarMass > MW_Low) & (MW_StellarMass < MW_High))
index_M31 = np.where((M31_StellarMass > M31_Low) & (M31_StellarMass < M31_High))

ReMW = r[index_MW]
ReM31 = r[index_M31]

In [31]:
#405 snap value this and below through 30
print(MW_StellarTotal)
print(M31_StellarTotal)

71986000000.00003 solMass
112664000000.00003 solMass


In [32]:
print(MW_Low)
print(MW_High)
print(M31_Low)
print(M31_High)

35993000000.000015 solMass
36352930000.000015 solMass
56332000000.000015 solMass
56895320000.000015 solMass


In [37]:
print(index_MW)
print(index_M31)

(array([], dtype=int64),)
(array([68, 69]),)


In [30]:
print(ReMW)
print(ReM31)

[]
[6.9 7. ]


In [20]:
#800 snap value this and below through 24
print(MW_StellarTotal)

59748000000.000015 solMass


In [21]:
print(MW_Low)
print(MW_High)

29874000000.000008 solMass
30172740000.000008 solMass


In [22]:
print(index_MW)

(array([68]),)


In [23]:
print(ReMW)

[6.9]


In [24]:
print(ReM31)

[8.6]


In [ ]:
#plotting with position on the left and sersic profile on the right
fig, ax = plt.subplots(nrows=1, ncols=2, figsize =(20 ,10)) 

ax[0].hist2d(MWx,MWy, bins=150, norm=LogNorm(),cmap='magma')
ax[0].hist2d(M31x,M31y, bins=150, norm=LogNorm(),cmap='magma' )
#plt.colorbar()
# Add axis labels
ax[0].set_xlabel('x (kpc)', fontsize=22)
ax[0].set_ylabel('y (kpc)', fontsize=22)
#set axis limits
ax[0].set_ylim(-200,200)
ax[0].set_xlim(-200,200)

#plot disk luminosity density- this is a proxy for surface density
ax[1].semilogy(r,MW_StellarI, color='blue',linewidth=3,label = "MW Disk Particles")
ax[1].semilogy(r,M31_StellarI, color='red', linewidth = 3, label = "M31 Disk Particles")
#ax[1].semilogy(r,Both_StellarI, color='purple', linewidth=3, label = "All particles")
#Sersic
ax[1].semilogy(r,SersicE(r, ReMW, 4, MW_StellarTotal), color = 'black', linestyle = '--',
            linewidth = 3, label = 'Sersic n=4, Elliptical')

ax[1].semilogy(r,SersicE(r, ReMW, 1, MW_StellarTotal), color = 'black', linestyle = ':',
            linewidth = 3, label = 'Sersic n=1, Spiral')
# Add axis labels
ax[1].set_xlabel('Radius (kpc)', fontsize=22)
ax[1].set_ylabel('Log(I)  $L_\odot/kpc^2$', fontsize=22)
#limit on the yscale
ax[1].set_ylim(1e5,1e11)

legend = ax[1].legend(loc='upper right',fontsize='x-large')

fig.savefig("Sample_Plot.png")